In [1]:
from keras import backend as K
from keras.models import load_model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
model = load_model('models/batchnorm_ft.h5')

In [35]:
path = '/home/ubuntu/nbs/state_farm/data/'
batch_size=64

In [36]:
from keras.preprocessing.image import ImageDataGenerator
def get_batches(directory, target_size=(224, 224), batch_size=1, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

test_batches = get_batches(path+'test', batch_size=batch_size)


Found 79726 images belonging to 1 classes.


In [37]:
preds = model.predict_generator(test_batches, steps=test_batches.samples//batch_size + 1)
print(preds.shape)

(79726, 10)


In [38]:
print(test_batches.samples//batch_size)

1245


In [39]:
print(path)

/home/ubuntu/nbs/state_farm/data/


In [40]:
preds = preds.clip(min=0.02, max=0.98)

In [41]:
import numpy as np
filenames = test_batches.filenames
ids = np.array([f[8:] for f in filenames])

In [42]:
import pandas as pd
#subm = np.ndarray.transpose(subm)
df_subm = pd.DataFrame(preds, index=ids, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
#df_subm['img'] = df_subm.index
df_subm.reset_index(level=0, inplace=True)
df_subm=df_subm.rename(columns = {'index':'img'})
df_subm.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.02000,0.02,0.02,0.02,0.020000,0.020000,0.973116,0.020000,0.02000,0.020000
1,img_14887.jpg,0.02000,0.02,0.02,0.02,0.020000,0.544510,0.020000,0.035394,0.02000,0.402385
2,img_62885.jpg,0.02000,0.02,0.02,0.02,0.980000,0.020000,0.020000,0.020000,0.02000,0.020000
3,img_45125.jpg,0.02000,0.02,0.02,0.02,0.020000,0.020000,0.645520,0.166659,0.15356,0.020000
4,img_22633.jpg,0.11833,0.02,0.02,0.02,0.294534,0.461856,0.020000,0.020000,0.02000,0.083869


In [43]:
from IPython.display import FileLink
submission_file_name = 'submission.csv.gz'
df_subm.to_csv(submission_file_name, index=False, compression='gzip')

In [44]:
FileLink(submission_file_name)

/home/ubuntu/nbs/state_farm/submission.csv.gz

In [49]:
rank = 473
total = 1441.0

In [50]:
rank/total*100.0

32.82442748091603